In [5]:
EXPT_NAME="Exp02: Shell and Tube Condenser"
import pandas as pd
import numpy as np
import math

import platform
from datetime import datetime
print("**************************************")
print(EXPT_NAME)
print("Roll No: 120CH0057, Name: Siddharth Gautam")
print("Running On: Python ", platform.python_version(), " on: ", str(datetime.now()))
print("**************************************")
print("")

# 10000 kg/hr
MFR_steam = 10000/3600
T_steam = 120
T_sat = 100

# Properties at (T_steam + T_sat) / 2 = 110 C 
Cp_steam = 2180
Hvap_steam = 2257
mu_steam = 12.46 * (10 ** (-6))
rho_steam = 0.826
k_steam = 0.0184

t_ci = 20
t_co = 30
Cp_water = 4186
mu_water = 0.001
rho_water = 997
k_water = 0.598

t_av = (t_ci + t_co) * 0.5
T_av = (T_sat + T_steam) * 0.5

# Assume U = 100 Btu / lb ft^2 F = 567 W / m^2 K
U_assume = 567
# Length = 2.4m
tube_length = 2.4
outer_diameter = 0.0254                                                                                                                                                                                                                                                                     
pitch = 1.25 * 0.0254
# 1 in, 16 BWG
a_t = 0.00038322504 # 0.594 in^2 from DQ Kern Table 10
inner_diameter = 0.87 * 0.0254

print("Steam flow rate: ", MFR_steam, " kg/s")
# Step I : Heat Balances to get water flow rate
Q = MFR_steam * Cp_steam * (T_steam - T_sat) + MFR_steam * Hvap_steam
print("Net Heat Transfer: " + str(Q) + " J/s")

MFR_water = Q / ((t_co - t_ci) * Cp_water)
print("Water Flow Rate: " + str(MFR_water) + " kg/s")

dT1 = T_steam - t_ci
dT2 = T_sat - t_co

LMTD = (dT1 - dT2) / math.log(dT1 / dT2)
print("LMTD: " + str(LMTD) + " C")
print("LMTD correction is not needed as hot fluid is on shell side.")

# Q = UA(LMTD)
A = Q/(U_assume*LMTD)
print("Area: " + str(A) + " m^2")

# N_tubes
N_tubes = A / (math.pi * outer_diameter * tube_length)
print("Number of tubes: " + str(N_tubes))

# From DQ Kern's Table 9, we have the closest value for 2P as 14 tubes and 8' shell diameter
shell_diameter = 0.2032
N_tubes = 14
A_new = N_tubes * (math.pi * outer_diameter * tube_length)
print("New Area: " + str(A_new) + " m^2")
U_corrected = Q / (A_new * LMTD)
print("U (corrected): " + str(U_corrected) + " W/m^2 K")

# Cold Fluid Side
a_tube = N_tubes * a_t / 2
print("Tube flow area: " + str(a_tube) + " m^2")

G_tube = MFR_water / a_tube
print("Tube mass velocity: ", G_tube, "kg/ m^2 s")

v_tube = G_tube / rho_water
print("Tube velocity: ", v_tube, " m/s")

Re_t = rho_water * v_tube * inner_diameter / mu_water
print("Reynolds Number (Tube Side): ", Re_t)

Pr_t = Cp_water * mu_water / k_water 
print("Prandtl Number (Tube Side): ", Pr_t)

h_i = (Re_t ** 0.8)*(Pr_t ** 0.4)*0.23
print("Tube Side HTC: ", h_i, " W/m^2 K")

# Hot Fluid Side
h_o_assume = 150
h_io = (h_i * (inner_diameter / outer_diameter))
print("h_io: ", h_io)

# Wall temperature
t_w = t_av + (h_o_assume / (h_io + h_o_assume))*(T_av - t_av)
print("Wall temperature: ", t_w, " C")
T_f = (t_w + T_av) * 0.5
print("Film temperature: ", T_f, " C")

# properties at film temp
rho_water_f = 997
mu_water_f = 0.000404
k_water_f = 0.659

D_b = outer_diameter * (N_tubes / 0.175)**(1/1.285)
print("D_b: ", D_b, "m")

v_shell = MFR_steam / (rho_steam * math.pi * D_b * D_b * 0.25)
print("v_shell: ", v_shell)

Re_s = rho_water_f * shell_diameter * v_shell / mu_water_f
print("Reynolds Number (Shell Side): ", Re_s)

N_r = D_b / pitch 
print("N_r", N_r)

tube_loading = MFR_steam / (tube_length * N_tubes)
print("Tube Loading: ", tube_loading, " kg m/s")

h_s = 0.95 * k_water_f * (((rho_water_f * rho_water_f * 9.81) / mu_water_f * tube_loading)**(1/3)) * (N_r ** (1/6))
print("Shell Side HTC: ", h_s, " W/m^2 K")

# Final HTC
U_final = 1/(1/(h_i) + (1/h_s))
print("Overall HTC (calculated): ", U_final)

# Dirt Factor
dirt_factor = (1/(U_assume)) - (1/U_final)
print("Dirt Factor", dirt_factor)

# Baffle Design
N_baffles = round(tube_length / (0.14 * shell_diameter))
baffle_spacing = shell_diameter / 5
print("Number of Baffles: ", N_baffles)
baffle_length = 0.75 * shell_diameter
print("Baffle Length", baffle_length, " m")

# Pressure Drop, Tube Side
j_ft = 3.8 * (10 **(-3)) # Fig 12.24, Chemical Engineering Design
dP_t = j_ft*2*8*(tube_length / outer_diameter)*(rho_water * v_tube * v_tube)*0.5
print("Tube-Side Pressure Drop: ", (dP_t/1000), " kPa")

# Pressure Drop, Shell Side
j_fs = 1.0 * (10 ** (-3))
dP_s = 8 * j_fs * (shell_diameter / D_b) * (N_baffles + 1) * rho_water * v_shell * v_shell * 0.5
print("Shell-Side Pressure Drop", dP_s/1000, "kPa")

result = { "LMTD (C)": LMTD, "Tube Side HTC (W/m^2 K)": h_i, "Shell Side HTC (W/m^2 K)": h_s, "Number of Tubes": N_tubes, "Final U (W/m^2 K)": U_final, "Number of Baffles": N_baffles, "Baffle Spacing": baffle_spacing, "Tube Side Pressure Drop (kPa)": dP_t, "Shell Side Pressure Drop (kPa)": dP_s }
df = pd.DataFrame(result.items(), columns=["Parameter", "Value"])
display(df)

**************************************
Exp02: Shell and Tube Condenser
Roll No: 120CH0057, Name: Siddharth Gautam
Running On: Python  3.11.1  on:  2023-04-25 15:35:01.200429
**************************************

Steam flow rate:  2.7777777777777777  kg/s
Net Heat Transfer: 127380.55555555555 J/s
Water Flow Rate: 3.0430137495354885 kg/s
LMTD: 84.11019756171387 C
LMTD correction is not needed as hot fluid is on shell side.
Area: 2.6709848625315904 m^2
Number of tubes: 13.946864953921562
New Area: 2.681160834279673 m^2
U (corrected): 564.8480306338233 W/m^2 K
Tube flow area: 0.00268257528 m^2
Tube mass velocity:  1134.3628535693836 kg/ m^2 s
Tube velocity:  1.1377761821157308  m/s
Reynolds Number (Tube Side):  25067.150338176238
Prandtl Number (Tube Side):  7.0
Tube Side HTC:  1655.9645086731712  W/m^2 K
h_io:  1440.6891225456588
Wall temperature:  33.015393969373186  C
Film temperature:  71.50769698468659  C
D_b:  0.7688428812146471 m
v_shell:  7.243565230901324
Reynolds Number (Shell 

,Parameter,Value
0,LMTD (C),84.110198
1,Tube Side HTC (W/m^2 K),1655.964509
2,Shell Side HTC (W/m^2 K),1340.612892
3,Number of Tubes,14.000000
4,Final U (W/m^2 K),740.847665
5,Number of Baffles,84.000000
6,Baffle Spacing,0.040640
7,Tube Side Pressure Drop (kPa),3707.318883
8,Shell Side Pressure Drop (kPa),4700.725945
